In [2]:
!pip install webdriver-manager 

In [5]:
!pip install selenium

In [6]:
!pip install html5lib


PASSO 1: ESCOLHER O NAVEGADOR

Nesse projeto foi utilizado Microsoft Edge 



PASSO 2: IMPORTAR AS BIBLIOTECAS


In [2]:
from selenium import webdriver 
from selenium.webdriver.edge.service import Service as EdgeService
from webdriver_manager.microsoft import EdgeChromiumDriverManager
import time
import pandas as pd

#Faço o webdriver abrir o navegador e no caso um site específico

driver = webdriver.Edge(service = EdgeService(EdgeChromiumDriverManager().install()))

url = "https://www.etf.com/etfanalytics/etf-finder"

driver.get(url)

time.sleep(5)

botao_100 = driver.find_element("xpath", '''/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]
/section[2]/div[2]/section[2]/div[1]/div/div[4]/button/label/span''' )

driver.execute_script("arguments[0].click();", botao_100)

numero_paginas = driver.find_element("xpath", '''/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/
section[2]/div[2]/section[2]/div[2]/div/label[2]''')

numero_paginas = numero_paginas.text.replace("of ", "")

numero_paginas = int(numero_paginas)

lista_de_tabela_por_pagina = []

for pagina in range(0, numero_paginas):
    
    tabela = driver.find_element("xpath", '''/html/body/div[5]/section/div/div[3]/section/div/
                                    div/div/div/div[2]/section[2]/div[2]/div/table''')
    
    html_tabela = tabela.get_attribute("outerHTML")

    tabela_final = pd.read_html(html_tabela)[0]
    
    lista_de_tabela_por_pagina.append(tabela_final)
    
    botao_avancar_pagina = driver.find_element("xpath", '//*[@id="nextPage"]')
    
    driver.execute_script("arguments[0].click();", botao_avancar_pagina)
    
base_de_dados_completa = pd.concat(lista_de_tabela_por_pagina)
  

#primeira coisa é mudar a aba

botao_aba = driver.find_element( "xpath", '''/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/
                                          section[2]/div[2]/ul/li[2]/span''')

#comando que faz clicar no botao aba
driver.execute_script("arguments[0].click();", botao_aba)

#voltando a 1 página

for pagina in range(0, numero_paginas):
    
    botao_voltar_pagina = driver.find_element("xpath", '//*[@id="previousPage"]')
    
    driver.execute_script("arguments[0].click();", botao_voltar_pagina)

#Criando novamente o loop para copiar todas as paginas
lista_de_tabela_por_pagina = []

for pagina in range(0, numero_paginas):
    
    #encontra a tabela que queremos
    tabela = driver.find_element("xpath", '''/html/body/div[5]/section/div/div[3]/
                                          section/div/div/div/div/div[2]/section[2]/div[2]/div''')
    #extrai a tabela
    html_tabela = tabela.get_attribute("outerHTML")
    
    #Pandas lê e transforma em uma tabela bonitinha o dado extraído anteriormente
    tabela_final = pd.read_html(html_tabela)[0]
    
    #adiciona todas as página em uma tabela só 
    lista_de_tabela_por_pagina.append(tabela_final)
    
    #Encontra o botão de avnçar
    botao_avancar_pagina = driver.find_element("xpath", '//*[@id="nextPage"]')
    
    #Exceuta o click o no botao avançar
    driver.execute_script("arguments[0].click();", botao_avancar_pagina)


base_de_dados_performance = pd.concat(lista_de_tabela_por_pagina)

driver.quit() 


#Fazemos a tabela iniciar da coluna Ticker e assim removemos a coluna indice
base_de_dados_completa = base_de_dados_completa.set_index("Ticker")


base_de_dados_performance = base_de_dados_performance.set_index("Ticker")
base_de_dados_performance = base_de_dados_performance[['1 Year', '5 Years', '10 Years']]

base_de_dados_final = base_de_dados_completa.join(base_de_dados_performance)

display(base_de_dados_completa)
display(base_de_dados_performance)

base_de_dados_final

,Name,Segment,Issuer,Expense Ratio,AUM
Ticker,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$376.42B
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$307.48B
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$281.55B
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$279.95B
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$161.33B
...,...,...,...,...,...
CAMX,Cambiar Aggressive Value ETF,Equity: Global - Total Market,NaN,--,--
CVLC,Calvert US Large-Cap Core Responsible Index ETF,Equity: U.S. - Large Cap,Morgan Stanley,--,--
CVMC,Calvert US Mid-Cap Core Responsible Index ETF,Equity: U.S. - Mid Cap,Morgan Stanley,0.15%,--


,1 Year,5 Years,10 Years
Ticker,,,
SPY,-7.71%,11.19%,12.47%
IVV,-7.72%,11.21%,12.52%
VTI,-8.34%,10.69%,12.09%
VOO,-7.73%,11.22%,12.52%
QQQ,-15.77%,14.74%,17.07%
...,...,...,...
CAMX,--,--,--
CVLC,--,--,--
CVMC,--,--,--


,Name,Segment,Issuer,Expense Ratio,AUM,1 Year,5 Years,10 Years
Ticker,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$376.42B,-7.71%,11.19%,12.47%
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$307.48B,-7.72%,11.21%,12.52%
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$281.55B,-8.34%,10.69%,12.09%
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$279.95B,-7.73%,11.22%,12.52%
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$161.33B,-15.77%,14.74%,17.07%
...,...,...,...,...,...,...,...,...
CAMX,Cambiar Aggressive Value ETF,Equity: Global - Total Market,NaN,--,--,--,--,--
CVLC,Calvert US Large-Cap Core Responsible Index ETF,Equity: U.S. - Large Cap,Morgan Stanley,--,--,--,--,--
CVMC,Calvert US Mid-Cap Core Responsible Index ETF,Equity: U.S. - Mid Cap,Morgan Stanley,0.15%,--,--,--,--
